In [2]:
import xarray
import dask
import netCDF4
from pyhdf.SD import SD
import numpy
import pystare

In [3]:
file_path = '/home/griessbaum/MOD09/MOD09.A2019317.0815.006.2019319020759.hdf'
hdf = SD(file_path)
lon = hdf.select('Longitude').get().astype(numpy.double)
lat = hdf.select('Latitude').get().astype(numpy.double)

In [4]:
lat_x = xarray.DataArray(lat, dims=['x', 'y']).chunk({'x': 500})
lon_x = xarray.DataArray(lon, dims=['x', 'y']).chunk({'x': 500})

s_d = xarray.apply_ufunc(pystare.from_latlon2D, 
                         lat_x,
                         lon_x,
                         dask='parallelized',
                         output_dtypes=[numpy.int64])

sids = numpy.array(s_d)

# Write NetCDF

In [5]:
def write_variable():
    sids_netcdf = rootgrp.createVariable(varname='STARE_index_5km', 
                                     datatype='u8', 
                                     dimensions=('i_1km', 'j_1km'),
                                     chunksizes=[i_1km, j_1km])
    

i_1km = lon.shape[0]
j_1km = lon.shape[1]
l_1km = i_1km * j_1km # this is wrong!

with netCDF4.Dataset('test.nc', "w", format="NETCDF4") as rootgrp:
    pass
    
with netCDF4.Dataset('test.nc', "a", format="NETCDF4") as rootgrp:
    rootgrp.createDimension("i_1km", i_1km)
    rootgrp.createDimension("j_1km", j_1km)
    rootgrp.createDimension("l_1km", i_1km*j_1km)

with netCDF4.Dataset('test.nc', "a", format="NETCDF4") as rootgrp:
    lats_netcdf = rootgrp.createVariable(varname='Latitude_1km', 
                                         datatype='f8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    lats_netcdf.long_name = 'latitude'
    lats_netcdf.units = 'degrees_north'
    lats_netcdf[:, :] = lat

with netCDF4.Dataset('test.nc', "a", format="NETCDF4") as rootgrp:
    lons_netcdf = rootgrp.createVariable(varname='Longitude_1km',
                                         datatype='f8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    lons_netcdf.long_name = 'longitude'
    lons_netcdf.units = 'degrees_east'
    lons_netcdf[:, :] = lon
    

with netCDF4.Dataset('test.nc', 'a', format="NETCDF4") as rootgrp:
    cover_netcdf = rootgrp.createVariable(varname='STARE_cover_1km', 
                                          datatype='u8', 
                                          dimensions=('l_1km'),
                                          chunksizes=[l_1km])
    cover_netcdf.long_name = 'SpatioTemporal Adaptive Resolution Encoding (STARE) cover'

with netCDF4.Dataset('test.nc', "a", format="NETCDF4") as rootgrp:
    sids_netcdf = rootgrp.createVariable(varname='STARE_index_1km', 
                                         datatype='u8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    sids_netcdf.long_name = 'SpatioTemporal Adaptive Resolution Encoding (STARE) index'
    sids_netcdf[:, :] = sids


# Use the modules

## Mod09

In [4]:
import importlib
import sys
sys.path.insert(0,'../') 

In [5]:
import products
importlib.reload(products)

<module 'products' from '../products/__init__.py'>

In [6]:
file_path = '/home/griessbaum/MOD09/MOD09.A2019317.0815.006.2019319020759.hdf'
granule = products.MOD09(file_path)

In [7]:
granule.read_gring()
granule.read_laton()

In [8]:
chunk_size = 500
lat_x = xarray.DataArray(lat, dims=['x', 'y']).chunk({'x': chunk_size})
lon_x = xarray.DataArray(lon, dims=['x', 'y']).chunk({'x': chunk_size})
with dask.distributed.Client(n_workers=4) as client:
    sids = xarray.apply_ufunc(pystare.from_latlon2D,
                              lat_x, 
                              lon_x,
                              kwargs={'adapt_resolution': True},
                              dask='parallelized',                          
                              output_dtypes=[numpy.int64])
sids = numpy.array(sids)

In [15]:
cover_res = int(pystare.spatial_resolution(sids).max())
cover_sids = pystare.to_nonconvex_hull_range_from_latlon(granule.gring_lats,
                                                         granule.gring_lats,
                                                         13)

## Sidecar

In [12]:
import sidecar
importlib.reload(sidecar)

<module 'sidecar' from '../sidecar.py'>

In [13]:
scar = sidecar.Sidecar(file_path)

In [16]:
i = sids.shape[0]
j = sids.shape[1]
l = cover_sids.size 
scar.write_dimensions(i, j, l)

In [21]:
granule.lons.shape[0]

2030

In [18]:
scar.write_lons(granule.lons, nom_res='1km')
scar.write_lats(granule.lats, nom_res='1km')
scar.write_sids(sids, nom_res='1km')
scar.write_cover(cover_sids, nom_res='1km')

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [11]:
granule.lons

array([[24.93397331, 24.97850037, 25.02305412, ..., 45.98316574,
        46.02572632, 46.06861877],
       [24.93208122, 24.97674942, 25.02131081, ..., 45.97977448,
        46.02232361, 46.06520462],
       [24.93017769, 24.97513962, 25.02020073, ..., 45.97639847,
        46.01893616, 46.0617981 ],
       ...,
       [21.30373573, 21.3474369 , 21.38975334, ..., 42.03189468,
        42.07458496, 42.11760712],
       [21.30073547, 21.34402466, 21.38656235, ..., 42.02973938,
        42.07242203, 42.11544037],
       [21.29780769, 21.34095573, 21.38399315, ..., 42.02759171,
        42.07027435, 42.11328888]])